# Fastest Code

In [ ]:
import pandas as pd
    def find_cluster(cluster_name, dataframe):
        """
       returns clusters with disjoint maxpaths annotated
       to different genomes or annotated on the same
       genome min of 10kb apart
        """
        cluster = cluster_name
        df = dataframe
        maxpaths = [set(item) for item in [_.split(',') for _ in df['maxpath'].as_matrix()]]
        chrom_string = df['chrom_pos'].as_matrix()
        chrom = [_.split(':')[0] for _ in df['chrom_pos'].as_matrix()]
        chrom_end = [int(_.split('-')[1]) for _ in df['chrom_pos'].as_matrix()]
        for idx,item in enumerate(maxpaths):
            for j in range(idx+1, len(maxpaths)):
                if item.isdisjoint(maxpaths[j]):
                    if chrom[idx] != chrom[j]:
                        return(cluster)
                    elif chrom[idx] == chrom[j]:
                        if abs(chrom_end[idx] - chrom_end[j]) > 10000:
                            return(cluster)

    def df_to_dict(dataframe):
        """ 
        Create dict with cluster as key and
        subset of dataframe at cluster as value
        """
        df = dataframe
        unique_clusters = df['cluster_name'].unique()
        sub_dfs = []
        for i in unique_clusters:
            sub_dfs.append(df[df['cluster_name'] == i])
        return(dict(zip(unique_clusters, sub_dfs)))

    def get_clusters(dataframe):
        """
        return disjoint clusters
        """
        df = pd.read_csv(dataframe, index_col=False)
        df_dict = df_to_dict(df)
        disjoint = [find_cluster(k,v) for k,v in df_dict.items() if find_cluster(k,v)]
        return(disjoint)

In [ ]:
x = get_clusters('results/disjoint_maxpaths_greater_than_one.csv')
df = pd.DataFrame({'clusters_with_disjoint_maxpaths':x})
df.to_csv('clusters_with_disjoint_maxpaths.csv', index=False)
print(x)

# Some fo the rough drafts of the code below. For the curious

# Testing 2 ways of trimming maxpaths

In [86]:
new_df = untouched_df.groupby('cluster_name').agg('count')
new_df = new_df[new_df['maxpath'] > 1]
clusters = new_df.index
abundance_df = pd.read_csv('/mnt/remote/SQLite/results/disjoint_maxpaths_abundance.csv')
trimmed_df = abundance_df[abundance_df.groupby('cluster_name').cluster_name.transform(len) > 1]
trimmed_df



Index(['cluster_0', 'cluster_1', 'cluster_10', 'cluster_100', 'cluster_1000',
       'cluster_10000', 'cluster_100004', 'cluster_100007', 'cluster_100009',
       'cluster_10001',
       ...
       'cluster_99921', 'cluster_9993', 'cluster_9994', 'cluster_9995',
       'cluster_99961', 'cluster_99969', 'cluster_9997', 'cluster_99973',
       'cluster_9999', 'cluster_99991'],
      dtype='object', name='cluster_name', length=68656)

# Difference in speed between old and new find clusters

In [40]:
def find_cluster(cluster_name, dataframe):
    """
   returns clusters with disjoint maxpaths annotated
   to different genomes or annotated on the same
   genome min of 10kb apart
    """
    cluster = cluster_name
    df = dataframe
    maxpaths = [set(item) for item in [_.split(',') for _ in synth_df['maxpaths'].as_matrix()]]
    chrom_string = df['chrom_pos'].as_matrix()
    chrom = [_.split(':')[0] for _ in df['chrom_pos'].as_matrix()]
    chrom_end = [int(_.split('-')[1]) for _ in df['chrom_pos'].as_matrix()]
    for idx,item in enumerate(maxpaths):
        for j in range(idx+1, len(maxpaths)):
            if item.isdisjoint(maxpaths[j]):
                if chrom[idx] != chrom[j]:
                    return(cluster)
                elif chrom[idx] == chrom[j]:
                    if abs(chrom_end[idx] - chrom_end[j]) > 10000:
                        return(cluster)

def old_find_cluster(cluster, maxpaths):
    for idx, item in enumerate(maxpaths):
        unique = set(item.split(','))
        for j in range(idx+1, len(maxpaths)):
            compare = set(maxpaths[j].split(','))
            chrom_string1 = subset_df[
                subset_df['maxpath'] == item]['chrom_pos'].values[0]
            chrom_string2 = subset_df[
                subset_df['maxpath'] == maxpaths[j]]['chrom_pos'].values[0]

            chrom1 = chrom_string1.split(':')[0]
            chrom_end1 = int(chrom_string1.split('-')[1])

            chrom2 = chrom_string2.split(':')[0]
            chrom_end2 = int(chrom_string2.split('-')[1])
            if len(unique.intersection(compare)) == 0:
                if chrom1 != chrom2:
                    return(cluster)
                elif chrom1 == chrom2:
                    if abs(chrom_end1 - chrom_end2) > 10000:
                        return(cluster)

                        
def test_new():
    cluster = ["cluster_689"]
    disjoint_cluster = []
    for i in cluster:
        found = find_cluster(i, subset_df)
        disjoint_cluster.append(found)
    return(disjoint_cluster)

def test_old():
    cluster = ["cluster_689"]
    disjoint_cluster = []
    for i in cluster:
        maxpaths_array = subset_df['maxpath'].as_matrix()
        found = old_find_cluster(i,maxpaths_array)
        disjoint_cluster.append(found)
    return(disjoint_cluster)

new = %timeit for x in range(3): test_new()
old = %timeit for x in range(3): test_old()


247 µs ± 5.48 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
138 ms ± 587 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
unique_clusters = trimmed_df['cluster_name'].unique()[0:10]
sub_dfs = []
for i in unique_clusters:
    sub_dfs.append(trimmed_df[trimmed_df['cluster_name'] == i])
cluster_df_dict = dict(zip(unique_clusters, sub_dfs))



In [6]:
cluster_df_dict.keys()

dict_keys(['cluster_100611', 'cluster_100742', 'cluster_1007', 'cluster_1027', 'cluster_10311', 'cluster_105119', 'cluster_10522', 'cluster_10641', 'cluster_106862', 'cluster_106908'])

In [7]:
len(cluster_df_dict.keys())

10

# NEed to add an if statement

In [14]:
disjoint = [find_cluster(k,v) for k,v in cluster_df_dict.items() if find_cluster(k,v)]
disjoint

['cluster_1027', 'cluster_10311', 'cluster_10641']

In [15]:
    def find_cluster(cluster_name, dataframe):
        """
       returns clusters with disjoint maxpaths annotated
       to different genomes or annotated on the same
       genome min of 10kb apart
        """
        cluster = cluster_name
        df = dataframe
        maxpaths = [set(item) for item in [_.split(',') for _ in df['maxpath'].as_matrix()]]
        chrom_string = df['chrom_pos'].as_matrix()
        chrom = [_.split(':')[0] for _ in df['chrom_pos'].as_matrix()]
        chrom_end = [int(_.split('-')[1]) for _ in df['chrom_pos'].as_matrix()]
        for idx,item in enumerate(maxpaths):
            for j in range(idx+1, len(maxpaths)):
                if item.isdisjoint(maxpaths[j]):
                    if chrom[idx] != chrom[j]:
                        return(cluster)
                    elif chrom[idx] == chrom[j]:
                        if abs(chrom_end[idx] - chrom_end[j]) > 10000:
                            return(cluster)

    def df_to_dict(dataframe):
        """ 
        Create dict with cluster as key and
        subset of dataframe at cluster as value
        """
        df = dataframe
        unique_clusters = df['cluster_name'].unique()
        sub_dfs = []
        for i in unique_clusters:
            sub_dfs.append(df[df['cluster_name'] == i])
        return(dict(zip(unique_clusters, sub_dfs)))

    def get_clusters(dataframe):
        """
        return disjoint clusters
        """
        df = pd.read_csv(dataframe, index_col=False)
        df_dict = df_to_dict(df)
        disjoint = [find_cluster(k,v) for k,v in df_dict.items() if find_cluster(k,v)]
        return(disjoint)


In [18]:
len(x)

9126